# GoWork opinions

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import csv

In [2]:
sourceURL = 'https://www.gowork.pl/opinie_czytaj,76776' # this is url for the first page of opinions for PwC
keys = ['Opinion', 'Timestamp'] # these will be our columns' names

In [3]:
def read_url(url):
    response = requests.get(url)
    return response.text

### Let's try it first for PwC to get insights into website's structure.

In [26]:
read_url(sourceURL)[:500] # this makes a request and gets text from the page

'    <!DOCTYPE html><html lang="pl" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb#"><head data-loggedin="0"><title>        Opinie PwC Polska Warszawa\n     - GoWork.pl</title><meta charset="utf-8" /><meta http-equiv="X-UA-Compatible" content="IE=edge" /><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no" /><meta http-equiv="Content-Language" content="pl" /><meta name="keywords" property="keywords" content="opinie o PwC Polska Warszawa, zarobki PwC Polska W'

In [5]:
response = read_url(sourceURL)
soup = BeautifulSoup(markup=response, features='lxml')

In [7]:
opinions = soup.find_all('div', "js-thread thread-item clearfix") # finds all div tags with class given by the second argument

In [9]:
opinions[0].find('p', "js-review-text") # we take first opinion and have to get to its text by seeking for p tag with class 
# given by the second argument, note that we used 'find' instead of 'find_all' - this returns first found p tag with given class 
# which is our opinion; subsequent p tags within this class are comments of an opinion

    

<p class="js-review-text review__text review-content-0589defe-5385-4ba7-8d58-3fae4a47e6ce">Mamy połowę 2021 roku natomiast pensje w FCU stoją dalej w miejscu. Inflacja w Polsce na poziomie 5% (i być może jeszcze wzrośnie), wszystko drożeje w niebotycznym tempie. Tak naprawdę zarobki bez zmian od kilku lat. Wielokrotnie temat był poruszany podczas streamów z leaderami, jednak dalej bez echa. Pensja, która była w miarę ok w 2017 roku dzisiaj coraz mocniej odstaje od realiów rynkowych. PwC to przecież Big4 czyli marka sama w sobie, dlatego też powinna wyznaczać pewne standardy, między innymi jeśli chodzi o podejście do własnych pracowników. Nie ulega wątpliwości, że zatrudnieni tutaj specjaliści to starannie wyselekcjonowani ludzie, pełni profesjonalizmu i ze sporą wiedzą. PwC powinno stawiać na jakość także w tym aspekcie. Te minimum 1000zł netto podwyżki dla każdego szczebla to według mnie najlepszy sposób aby okazać wdzięczność ludziom, którzy przynoszą coraz większe zyski dla firmy. M

In [10]:
opinions[0].find('p', "js-review-text").text # # we have to get to the opinion itself, excluding tags and here it is

'Mamy połowę 2021 roku natomiast pensje w FCU stoją dalej w miejscu. Inflacja w Polsce na poziomie 5% (i być może jeszcze wzrośnie), wszystko drożeje w niebotycznym tempie. Tak naprawdę zarobki bez zmian od kilku lat. Wielokrotnie temat był poruszany podczas streamów z leaderami, jednak dalej bez echa. Pensja, która była w miarę ok w 2017 roku dzisiaj coraz mocniej odstaje od realiów rynkowych. PwC to przecież Big4 czyli marka sama w sobie, dlatego też powinna wyznaczać pewne standardy, między innymi jeśli chodzi o podejście do własnych pracowników. Nie ulega wątpliwości, że zatrudnieni tutaj specjaliści to starannie wyselekcjonowani ludzie, pełni profesjonalizmu i ze sporą wiedzą. PwC powinno stawiać na jakość także w tym aspekcie. Te minimum 1000zł netto podwyżki dla każdego szczebla to według mnie najlepszy sposób aby okazać wdzięczność ludziom, którzy przynoszą coraz większe zyski dla firmy. Myślę, że wiele osób się ze mną zgodzi. Pokażmy ilu nas jest!'

In [11]:
# also datetime can be useful for futher analysis, so let's get it

opinions[0].find('time', 'review__date')

<time class="review__date" datetime="02.06.2021 11:39" title="02.06.2021 11:39">

                02.06.2021 11:39
            </time>

In [12]:
opinions[0].find('time', 'review__date').get('datetime') # that's how we get attribute's value

'02.06.2021 11:39'

In [13]:
date = '29.12.2020 22:16'
date

'29.12.2020 22:16'

In [14]:
import pandas as pd # that's how we can make our date string a proper timestamp to pandas 
pd.to_datetime(date, format="%d.%m.%Y %H:%M")

Timestamp('2020-12-29 22:16:00')

### Now let's try our process for a couple of opinions

In [15]:
for opinion in opinions[:5]:
    contents = opinion.find('p', 'js-review-text').text
    date = opinion.find('time', 'review__date').get('datetime')
    timestamp = pd.to_datetime(date, format="%d.%m.%Y %H:%M")
    print(contents, timestamp)
    print('__________')

Mamy połowę 2021 roku natomiast pensje w FCU stoją dalej w miejscu. Inflacja w Polsce na poziomie 5% (i być może jeszcze wzrośnie), wszystko drożeje w niebotycznym tempie. Tak naprawdę zarobki bez zmian od kilku lat. Wielokrotnie temat był poruszany podczas streamów z leaderami, jednak dalej bez echa. Pensja, która była w miarę ok w 2017 roku dzisiaj coraz mocniej odstaje od realiów rynkowych. PwC to przecież Big4 czyli marka sama w sobie, dlatego też powinna wyznaczać pewne standardy, między innymi jeśli chodzi o podejście do własnych pracowników. Nie ulega wątpliwości, że zatrudnieni tutaj specjaliści to starannie wyselekcjonowani ludzie, pełni profesjonalizmu i ze sporą wiedzą. PwC powinno stawiać na jakość także w tym aspekcie. Te minimum 1000zł netto podwyżki dla każdego szczebla to według mnie najlepszy sposób aby okazać wdzięczność ludziom, którzy przynoszą coraz większe zyski dla firmy. Myślę, że wiele osób się ze mną zgodzi. Pokażmy ilu nas jest! 2021-06-02 11:39:00
__________

### Now let's make trial csv file that will store our opinions from the first page for PWC.

In [83]:
file = open('opinions.csv', mode='w', newline='', encoding = 'utf-8')

In [84]:
writer = csv.writer(file)
writer # we instantiate csv writer

In [85]:
writer.writerow(keys) # columns' names are written 

for opinion in opinions:
    contents = opinion.find('p', 'js-review-text').text
    date = opinion.find('time', 'review__date').get('datetime')
    timestamp = pd.to_datetime(date, format="%d.%m.%Y %H:%M")
    writer.writerow([contents, timestamp])
file.close()

In [86]:
pd.read_csv('opinions.csv') # everything seems to work properly

,Opinion,Timestamp
0,"Pracuję w tej firmie już całkiem sporo czasu, ...",2020-12-29 22:16:00
1,Głoszone na „prawo i lewo” wartości PWC to jak...,2021-03-20 11:44:00
2,Mamy połowę 2021 roku natomiast pensje w FCU s...,2021-06-02 11:39:00
3,Pracuje w PwC 2 lata i uważam że jest to w por...,2021-06-28 23:05:00
4,"nareszcie, dzis byla info o podwyzkach i bonus...",2020-10-23 10:11:00
5,Nieprofesjonalny przebieg rekrutacji:\nSkładał...,2021-06-17 12:43:00
6,Nie polecam.\nCały przebieg praktyk strasznie ...,2021-06-09 17:48:00
7,"Pracuje w Wawie w Advisory - dużo pracy, ale p...",2020-03-31 11:47:00
8,Firma bardzo nieslowno. Mimo obiecanej odpowie...,2021-05-16 13:49:00
9,Mam pytanie o dział outsourcingu księgowego w ...,2021-05-09 11:19:00


In [96]:
# now we have to take opinions from all pages so we need links to those pages

endings = [''] + [','+str(i) for i in range(2, 21)]
endings

['',
 ',2',
 ',3',
 ',4',
 ',5',
 ',6',
 ',7',
 ',8',
 ',9',
 ',10',
 ',11',
 ',12',
 ',13',
 ',14',
 ',15',
 ',16',
 ',17',
 ',18',
 ',19',
 ',20']

In [103]:
pages = [sourceURL + end for end in endings]
pages

['https://www.gowork.pl/opinie_czytaj,76776',
 'https://www.gowork.pl/opinie_czytaj,76776,2',
 'https://www.gowork.pl/opinie_czytaj,76776,3',
 'https://www.gowork.pl/opinie_czytaj,76776,4',
 'https://www.gowork.pl/opinie_czytaj,76776,5',
 'https://www.gowork.pl/opinie_czytaj,76776,6',
 'https://www.gowork.pl/opinie_czytaj,76776,7',
 'https://www.gowork.pl/opinie_czytaj,76776,8',
 'https://www.gowork.pl/opinie_czytaj,76776,9',
 'https://www.gowork.pl/opinie_czytaj,76776,10',
 'https://www.gowork.pl/opinie_czytaj,76776,11',
 'https://www.gowork.pl/opinie_czytaj,76776,12',
 'https://www.gowork.pl/opinie_czytaj,76776,13',
 'https://www.gowork.pl/opinie_czytaj,76776,14',
 'https://www.gowork.pl/opinie_czytaj,76776,15',
 'https://www.gowork.pl/opinie_czytaj,76776,16',
 'https://www.gowork.pl/opinie_czytaj,76776,17',
 'https://www.gowork.pl/opinie_czytaj,76776,18',
 'https://www.gowork.pl/opinie_czytaj,76776,19',
 'https://www.gowork.pl/opinie_czytaj,76776,20']

## Now we can move on to gather entire data for PwC.

In [143]:
file = open('opinions_PwC.csv', mode='w', newline='', encoding='utf-8')
writer = csv.writer(file)

In [145]:
writer.writerow(keys)
for page in pages:
    response = read_url(page)
    soup = BeautifulSoup(markup = response, features='lxml')
    opinions = soup.find_all('div', 'js-thread thread-item clearfix')
    for opinion in opinions:
        contents = opinion.find('p', 'js-review-text').text
        date = opinion.find('time', "review__date").get('datetime')
        timestamp = pd.to_datetime(date, format= "%d.%m.%Y %H:%M")
        writer.writerow([contents, timestamp])
file.close()

In [146]:
pd.read_csv('opinions_PwC.csv')

,Opinion,Timestamp
0,"Pracuję w tej firmie już całkiem sporo czasu, ...",2020-12-29 22:16:00
1,Głoszone na „prawo i lewo” wartości PWC to jak...,2021-03-20 11:44:00
2,Mamy połowę 2021 roku natomiast pensje w FCU s...,2021-06-02 11:39:00
3,Pracuje w PwC 2 lata i uważam że jest to w por...,2021-06-28 23:05:00
4,"nareszcie, dzis byla info o podwyzkach i bonus...",2020-10-23 10:11:00
...,...,...
301,Świetne miejsce do poznania naprawdę bystrych ...,2014-10-16 15:31:00
302,"szczera prawda, jesteś częścią wielkiek maszyn...",2011-01-21 14:13:00
303,ogólna psychoza w otoczce szczęśliwości - korp...,2010-12-02 13:30:00
304,ludzie jak szczury / praca do utraty sił / ...,2010-12-02 13:29:00


# EY GoWork opinions

In [153]:
ey_url = 'https://www.gowork.pl/opinie_czytaj,4506'
endings = [''] + [','+ str(i) for i in range(2,15)]
pages = [ey_url + end for end in endings]
pages

['https://www.gowork.pl/opinie_czytaj,4506',
 'https://www.gowork.pl/opinie_czytaj,4506,2',
 'https://www.gowork.pl/opinie_czytaj,4506,3',
 'https://www.gowork.pl/opinie_czytaj,4506,4',
 'https://www.gowork.pl/opinie_czytaj,4506,5',
 'https://www.gowork.pl/opinie_czytaj,4506,6',
 'https://www.gowork.pl/opinie_czytaj,4506,7',
 'https://www.gowork.pl/opinie_czytaj,4506,8',
 'https://www.gowork.pl/opinie_czytaj,4506,9',
 'https://www.gowork.pl/opinie_czytaj,4506,10',
 'https://www.gowork.pl/opinie_czytaj,4506,11',
 'https://www.gowork.pl/opinie_czytaj,4506,12',
 'https://www.gowork.pl/opinie_czytaj,4506,13',
 'https://www.gowork.pl/opinie_czytaj,4506,14']

In [154]:
file_ey = open('opinions_EY.csv', mode='w', newline='', encoding = 'utf-8')
writer_ey = csv.writer(file_ey)
writer_ey.writerow(keys)
for page in pages:
    response = read_url(page)
    soup = BeautifulSoup(markup=response, features='lxml')
    opinions = soup.find_all('div', 'js-thread thread-item clearfix')
    for opinion in opinions:
        contents = opinion.find('p', 'js-review-text').text
        date = opinion.find('time', 'review__date').get('datetime')
        timestamp = pd.to_datetime(date, format = "%d.%m.%Y %H:%M")
        writer_ey.writerow([contents, timestamp])
file_ey.close()

In [155]:
pd.read_csv('opinions_EY.csv')

,Opinion,Timestamp
0,Nie polecam. Pracowałem w tej firmie prawie 5 ...,2019-04-21 20:08:00
1,"Hej, \npracuję w tej firmie od 2 lat na stanow...",2020-06-11 17:16:00
2,skoro Jan Krzysztof Bielecki ( były premier ) ...,2020-12-31 15:28:00
3,Przedstawiciel firmt podczas rozmowy starała s...,2021-07-14 14:22:00
4,"firma wspaniała, tylko ludzie...",2021-06-09 18:35:00
...,...,...
193,Jeśli chodzi o prace w usłogach ksiegowych to ...,2013-12-06 23:45:00
194,potwierdzam to co zostało napisane powyżej prz...,2013-02-01 11:53:00
195,W tym przypadku to naiwny optymizm,2013-01-24 04:15:00
196,Lepiej zatrudniać byłych i obecnych agentów po...,2012-07-18 17:14:00


# KPMG GoWork opinions

In [168]:
kpmg_url = 'https://www.gowork.pl/opinie_czytaj,12557'
endings = [''] + [',' + str(i) for i in range(2, 8)]
kpmg_pages = [kpmg_url + end for end in endings]
kpmg_pages

['https://www.gowork.pl/opinie_czytaj,12557',
 'https://www.gowork.pl/opinie_czytaj,12557,2',
 'https://www.gowork.pl/opinie_czytaj,12557,3',
 'https://www.gowork.pl/opinie_czytaj,12557,4',
 'https://www.gowork.pl/opinie_czytaj,12557,5',
 'https://www.gowork.pl/opinie_czytaj,12557,6',
 'https://www.gowork.pl/opinie_czytaj,12557,7']

In [165]:
file_kpmg = open('opinions_KPMG.csv', mode='w', newline = '', encoding = 'utf-8')
writer_kpmg = csv.writer(file_kpmg)
writer_kpmg.writerow(keys)
for page in kpmg_pages:
    response = read_url(page)
    soup = BeautifulSoup(markup= response, features= 'lxml')
    opinions = soup.find_all('div', 'js-thread thread-item clearfix')
    for opinion in opinions:
        contents = opinion.find('p', 'js-review-text').text
        date = opinion.find('time', 'review__date').get('datetime')
        timestamp = pd.to_datetime(date, format = "%d.%m.%Y %H:%M")
        writer_kpmg.writerow([contents, timestamp])
file_kpmg.close()

In [167]:
pd.read_csv('opinions_KPMG.csv')

,Opinion,Timestamp
0,"To gratuluję Wam pracy w kpmg, ja po studiach ...",2016-03-17 15:53:00
1,"masakryczna atmosfera, donoszenie, plotki, kom...",2019-11-24 14:37:00
2,Bardzo fatalne podejście od HR brak kontaktu t...,2021-06-11 16:41:00
3,Jestem zażenowana przebiegiem rekrutacji w tej...,2021-05-25 13:06:00
4,KPMG podobnie jak pozostałe pseudofirmy z Big4...,2021-04-25 08:53:00
...,...,...
96,Bardzo zła atmosfera pracy. Właściwie zaczynaj...,2009-05-16 12:51:00
97,"Rzeczywiście można się wiele nauczyć, ale kosz...",2009-03-09 13:07:00
98,"można zdobyć bogate doświadczenie, niestety ni...",2009-03-09 13:04:00
99,"typowa korporacja, pracownik jest tylko trybik...",2009-03-09 13:02:00


# Deloitte GoWork opinions

In [169]:
deloitte_url = 'https://www.gowork.pl/opinie_czytaj,29417'
endings = [''] + [',' + str(i) for i in range(2, 9)]
deloitte_pages = [deloitte_url + end for end in endings]
deloitte_pages

['https://www.gowork.pl/opinie_czytaj,29417',
 'https://www.gowork.pl/opinie_czytaj,29417,2',
 'https://www.gowork.pl/opinie_czytaj,29417,3',
 'https://www.gowork.pl/opinie_czytaj,29417,4',
 'https://www.gowork.pl/opinie_czytaj,29417,5',
 'https://www.gowork.pl/opinie_czytaj,29417,6',
 'https://www.gowork.pl/opinie_czytaj,29417,7',
 'https://www.gowork.pl/opinie_czytaj,29417,8']

In [170]:
file_deloitte = open('opinions_Deloitte.csv', mode = 'w', newline = '', encoding = 'utf-8')
writer_deloitte = csv.writer(file_deloitte)
writer_deloitte.writerow(keys)
for page in deloitte_pages:
    response = read_url(page)
    soup = BeautifulSoup(markup= response, features= 'lxml')
    opinions = soup.find_all('div', 'js-thread thread-item clearfix')
    for opinion in opinions:
        contents = opinion.find('p', 'js-review-text').text
        date = opinion.find('time', 'review__date').get('datetime')
        timestamp = pd.to_datetime(date, format = "%d.%m.%Y %H:%M")
        writer_deloitte.writerow([contents, timestamp])
file_deloitte.close()

In [171]:
pd.read_csv('opinions_Deloitte.csv')

,Opinion,Timestamp
0,"Nadmuchany profesjonalizm, który nijak ma się ...",2020-07-14 15:21:00
1,"Troszeczkę śmieszą mnie frustraci poniżej, któ...",2017-10-25 21:59:00
2,Ja przepraszam ale czytam sobie tak tu te opin...,2020-05-07 06:22:00
3,"Omijać szerokim łukiem. Pensje są takie, że mo...",2021-01-18 09:37:00
4,Uczestniczyłem w ponad godzinnej rozmowie w sp...,2021-05-26 08:58:00
...,...,...
96,Czy w firmie przyjmują na aplikację na biegłeg...,2016-03-11 08:53:00
97,"a czy na staże, programy, praktyki przyjmują s...",2016-02-26 18:51:00
98,Jak placa? Również interesuję mnie sprawa z ur...,2015-10-31 00:27:00
99,Jest duży nabór ale to nie znaczy że dużo ludz...,2015-10-15 22:18:00


Note that sometimes people ask questions in those opinions although they have special type of posts for that. In order to filter out those questions, we could get rid of posts with question mark(s) which will be done in second notebook.